In [1]:
# Load & Structural Check
import pandas as pd
import numpy as np

sales_df = pd.read_csv("dataset/Historical_Sales.csv")

sales_df.shape

(4243, 7)

In [2]:
sales_df.head()

,Date,SKU,Sessions - Total,Page Views - Total,Units Ordered,Ordered Product Sales,Total Order Items
0,2025-09-01,MN-01,22.0,32.0,6.0,228.10,5.0
1,2025-09-01,MN-02,28.0,33.0,5.0,158.96,5.0
2,2025-09-01,MN-03,9.0,12.0,2.0,81.35,2.0
3,2025-09-01,MN-07,31.0,41.0,2.0,70.11,2.0
4,2025-09-01,MN-09,24.0,32.0,2.0,32.26,1.0


In [3]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4243 entries, 0 to 4242
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   4243 non-null   object 
 1   SKU                    4243 non-null   object 
 2   Sessions - Total       4239 non-null   float64
 3   Page Views - Total     4239 non-null   float64
 4   Units Ordered          4239 non-null   float64
 5   Ordered Product Sales  4239 non-null   float64
 6   Total Order Items      4239 non-null   float64
dtypes: float64(5), object(2)
memory usage: 232.2+ KB


In [4]:
sales_df.isna().sum()

Date                     0
SKU                      0
Sessions - Total         4
Page Views - Total       4
Units Ordered            4
Ordered Product Sales    4
Total Order Items        4
dtype: int64

In [5]:
(sales_df == 0).sum()


Date                      0
SKU                       0
Sessions - Total         23
Page Views - Total       23
Units Ordered             0
Ordered Product Sales     2
Total Order Items         0
dtype: int64

- Zero sales = no demand
- Zero sessions = visibility issue

In [6]:
sales_df["Date"] = pd.to_datetime(sales_df["Date"], errors="coerce")

sales_df[[
    "Sessions - Total",
    "Page Views - Total",
    "Units Ordered",
    "Ordered Product Sales",
    "Total Order Items"
]] = sales_df[[
    "Sessions - Total",
    "Page Views - Total",
    "Units Ordered",
    "Ordered Product Sales",
    "Total Order Items"
]].fillna(0)


In [7]:
# Conversion Rate (Core pricing signal)
sales_df["Conversion_Rate"] = (
    sales_df["Units Ordered"] / sales_df["Sessions - Total"]
).replace([float("inf")], 0).fillna(0)


In [8]:
# Revenue Efficiency
sales_df["Revenue_per_Session"] = (
    sales_df["Ordered Product Sales"] /
    sales_df["Sessions - Total"]
).replace([float("inf")], 0).fillna(0)

sku_rps = (
    sales_df
    .groupby("SKU")["Revenue_per_Session"]
    .mean()
    .sort_values()
)


In [9]:
### SKU-Level Demand Summary 
sku_summary = (
    sales_df
    .groupby("SKU")
    .agg({
        "Sessions - Total": "mean",
        "Units Ordered": "mean",
        "Conversion_Rate": "mean",
        "Revenue_per_Session": "mean"
    })
)


In [10]:
## SKU-level average conversion (decision view)
sku_conversion = (
    sales_df
    .groupby("SKU")["Conversion_Rate"]
    .mean()
    .sort_values()
)

sku_conversion.head(10), sku_conversion.tail(10)

(SKU
 MN-07    0.148909
 MN-21    0.150302
 MN-24    0.156611
 MN-13    0.158888
 MN-11    0.165052
 MN-44    0.173213
 MN-12    0.173416
 MN-43    0.174714
 MN-10    0.175707
 MN-49    0.190341
 Name: Conversion_Rate, dtype: float64,
 SKU
 MN-25    0.474459
 MN-40    0.515667
 MN-32    0.551416
 MN-47    0.553008
 MN-28    0.594477
 MN-27    0.620459
 MN-42    0.642939
 MN-34    0.653606
 MN-41    0.768300
 MN-26    0.859363
 Name: Conversion_Rate, dtype: float64)

- Bottom SKUs → highly price sensitive
- Top SKUs → can tolerate price increases

In [11]:
#Demand Stability
sku_demand_stats = (
    sales_df
    .groupby("SKU")["Units Ordered"]
    .agg(["mean", "std"])
)

sku_demand_stats["Demand_CV"] = (
    sku_demand_stats["std"] / sku_demand_stats["mean"]
).replace([float("inf")], 0)

sku_demand_stats.sort_values("Demand_CV").head()


,mean,std,Demand_CV
SKU,,,
MN-29,16.692308,7.796997,0.467101
MN-28,27.252747,13.111991,0.481125
MN-15,25.042105,12.532439,0.500455
MN-01,6.380435,3.485740,0.546317
MN-25,49.131868,27.455503,0.558813


| Demand CV | Meaning                         |
| --------- | ------------------------------- |
| Low       | Stable demand → safer pricing   |
| High      | Volatile demand → risky pricing |


In [12]:
# Traffic vs Demand Diagnosis (pricing vs visibility)

sku_summary["Traffic_Level"] = pd.qcut(
    sku_summary["Sessions - Total"], 4,
    labels=["Low", "Medium", "High", "Very High"]
)

sku_summary["Demand_Level"] = pd.qcut(
    sku_summary["Units Ordered"], 4,
    labels=["Low", "Medium", "High", "Very High"]
)


| Pattern                       | Pricing Insight               |
| ----------------------------- | ----------------------------- |
| High traffic + low demand     | Price too high / weak value   |
| Low traffic + high conversion | Visibility issue, not pricing |
| High traffic + high demand    | Pricing power exists          |


In [13]:
# Aggregate daily data to SKU-level time series
sku_daily = (
    sales_df
    .groupby(["SKU", "Date"])["Units Ordered"]
    .sum()
    .reset_index()
)

# Demand trend strength (mean vs volatility)
trend_stats = (
    sku_daily
    .groupby("SKU")["Units Ordered"]
    .agg(["mean", "std"])
)

trend_stats["Demand_Stability"] = trend_stats["std"] / trend_stats["mean"]


In [14]:
trend_stats["Demand_Stability"].sum()

35.02399081500719

- Low RPS + high traffic → price/value mismatch
- High RPS → premium positioning possible

In [15]:
#SKU-level pricing recommendations
sku_summary["Pricing_Action"] = "Hold"

sku_summary.loc[
    (sku_summary["Conversion_Rate"] > sku_summary["Conversion_Rate"].median()) &
    (sku_demand_stats["Demand_CV"] < sku_demand_stats["Demand_CV"].median()),
    "Pricing_Action"
] = "Increase Price"

sku_summary.loc[
    (sku_summary["Conversion_Rate"] < sku_summary["Conversion_Rate"].median()) &
    (sku_summary["Traffic_Level"].isin(["High", "Very High"])),
    "Pricing_Action"
] = "Review / Reduce Price"


In [16]:
sales_df["Demand_Level"] = pd.cut(
    sales_df["Units Ordered"],
    bins=[-1, 5, 20, 50, 1000],
    labels=["Very Low", "Low", "Medium", "High"]
)


In [17]:
sales_df["Demand_Level"].value_counts()

Demand_Level
Very Low    2161
Low         1579
Medium       442
High          61
Name: count, dtype: int64

In [18]:
# Traffic vs Conversion Pattern
sales_df["Traffic_Level"] = pd.cut(
    sales_df["Sessions - Total"],
    bins=[-1, 100, 500, 2000, 100000],
    labels=["Very Low", "Low", "Medium", "High"]
)

In [19]:
pd.crosstab(
    sales_df["Traffic_Level"],
    sales_df["Demand_Level"]
)

Demand_Level,Very Low,Low,Medium,High
Traffic_Level,,,,
Very Low,2160,1513,340,18
Low,1,65,102,43
Medium,0,1,0,0


In [20]:
# Flag low-conversion SKUs
sales_df["Low_Conversion_Flag"] = sales_df["Conversion_Rate"] < 0.05


In [21]:
sales_df["Low_Conversion_Flag"].value_counts()

Low_Conversion_Flag
False    4140
True      103
Name: count, dtype: int64

In [22]:
# Flag strong demand SKUs
sales_df["High_Demand_Flag"] = (
    (sales_df["Units Ordered"] > sales_df["Units Ordered"].median()) &
    (sales_df["Conversion_Rate"] > sales_df["Conversion_Rate"].median())
)

In [23]:
sales_df["High_Demand_Flag"].value_counts()

High_Demand_Flag
False    2816
True     1427
Name: count, dtype: int64

- IF High Conversion AND Stable Demand:
    Allow price increase (even if competitors are aggressive)
- ELSE IF High Traffic AND Low Conversion:
    Consider price reduction or promo
- ELSE:
    Hold price, address non-pricing issues


In [24]:
sku_summary["Low_Conversion_Flag"] = (
    sku_summary["Conversion_Rate"] <
    sku_summary["Conversion_Rate"].median()
)

sku_summary["High_Demand_Flag"] = (
    (sku_summary["Conversion_Rate"] >
     sku_summary["Conversion_Rate"].median()) &
    (sku_demand_stats["Demand_CV"] <
     sku_demand_stats["Demand_CV"].median())
)


In [25]:
sales_signals = sku_summary.join(
    sku_demand_stats["Demand_CV"]
)

sales_signals.head()


,Sessions - Total,Units Ordered,Conversion_Rate,Revenue_per_Session,Traffic_Level,Demand_Level,Pricing_Action,Low_Conversion_Flag,High_Demand_Flag,Demand_CV
SKU,,,,,,,,,,
MN-01,20.880435,6.380435,0.338511,14.344181,Medium,High,Increase Price,False,True,0.546317
MN-02,21.670588,5.070588,0.247945,9.089562,Medium,Medium,Hold,True,False,0.708324
MN-03,11.012821,2.512821,0.267250,10.506878,Low,Low,Hold,True,False,0.766278
MN-04,8.162162,2.581081,0.420147,13.113315,Low,Low,Hold,False,False,0.734005
MN-05,27.000000,4.802817,0.236872,5.953501,High,Medium,Review / Reduce Price,True,False,1.095334


In [28]:
sales_signals.reset_index().to_csv(
    "final_outputs/sales_signals.csv",
    index=False
)

## Key Insights from Historical Sales Analysis

- Demand response varies significantly across SKUs, indicating that pricing power is not uniform across the catalog.
- Several products exhibit high traffic but weak conversion, suggesting price sensitivity or value misalignment rather than demand scarcity.
- A subset of SKUs shows strong and stable conversion despite moderate traffic, indicating pricing power and suitability for controlled price increases.
- Demand stability is a critical constraint for pricing decisions; volatile-demand SKUs should not be subjected to aggressive pricing changes.
